In [1]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
from sklearn import metrics

In [2]:
cwd = os.getcwd()
current_folder = cwd.split('\\')[-1]
print(current_folder)
if current_folder != 'BFOR-516-Project':
    os.chdir('..')
    cwd = os.getcwd()
current_folder = cwd.split('\\')[-1]
print(current_folder)
cwd

models
BFOR-516-Project


'C:\\Users\\jacks\\Documents\\GitHub\\BFOR-516-Project'

In [3]:
data = pd.read_csv('data/datafm20.csv')

In [29]:
data[['Name', 'Club', 'Division', 'Based', 'CA', 'PA']]

,Name,Club,Division,Based,CA,PA
0,Lionel Messi,Barcelona,Spanish First Division,Spain (First Division),195,200
1,Cristiano Ronaldo,Juventus,Italian Serie A,Italy (Serie A),195,196
2,Kylian MbappÃ©,Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),181,196
3,Manuel Neuer,FC Bayern,Bundesliga,Germany (Bundesliga),176,193
4,Neymar,Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),186,190
...,...,...,...,...,...,...
144745,Yuan Jinxing,Unknown,Unknown,China,3,3
144746,Aizudin Abdul Rahman,BSRC,Bruneian Premier League,Brunei (Premier League),2,2
144747,Berlin Jean-Gilles,CS Monteuil,PremiÃ¨re Ligue de soccer du QuÃ©bec,Canada (PLSQ),2,2
144748,Frederik van Hauen,Skovshoved,2. Division,Denmark (2. Division),1,2


Take the top 25%, 15%, 10%, 5%, or 1% of CA

1. See if we can predict if a  player is in that top percentile 
2. See if we can predict that a player that is not in the top percentile for CA but does have a PA above so has the potential to join that top percentile

In [46]:
data['CA'].quantile(.99)

133.0

In [11]:
data['PA'].quantile(.75)

111.0

In [47]:
data_2_CA = data.loc[data['CA'] >= 133]

In [48]:
data_2_CA[['Name', 'Club', 'Division', 'Based', 'CA', 'PA']]

,Name,Club,Division,Based,CA,PA
0,Lionel Messi,Barcelona,Spanish First Division,Spain (First Division),195,200
1,Cristiano Ronaldo,Juventus,Italian Serie A,Italy (Serie A),195,196
2,Kylian MbappÃ©,Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),181,196
3,Manuel Neuer,FC Bayern,Bundesliga,Germany (Bundesliga),176,193
4,Neymar,Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),186,190
...,...,...,...,...,...,...
6828,Nils Petersen,SC Freiburg,Bundesliga,Germany (Bundesliga),134,134
6834,Oliver Norwood,Sheff Utd,English Premier Division,England (Premier Division),133,134
6935,Walter Montoya,Racing Club,Argentine Premier Division,Argentina (Premier Division),133,134
6951,AdriÃ¡n GonzÃ¡lez,MÃ¡laga,Spanish Second Division,Spain (Second Division),133,133


In [44]:
data_2_PA = data.loc[(data['PA'] >= 108) & (data['CA'] < 108)]

In [45]:
data_2_PA

,Name,Position,Club,Division,Based,Nation,Height,Weight,Age,Preferred Foot,...,Cmp,Com,Cmd,Bra,Bal,Ant,Agi,Agg,Aer,Acc
44,Michael Essien,"DM, M (C)",SÃ¤bail,Azeri Premier League,Azerbaijan (Premier League),GHA,178,84,36,Right,...,15,1,1,16,13,15,7,14,1,7
56,Naci ÃnÃ¼var,"M (C), AM (LC), ST (C)",Ajax,Eredivisie,Holland (Eredivisie),NED,168,60,16,Right,...,13,2,2,10,12,10,14,12,3,11
59,Rayan Cherki,"AM (RLC), ST (C)",OL,Ligue 1 Conforama,France (Ligue 1 Conforama),FRA,177,71,15,Either,...,12,1,1,8,10,8,11,6,2,10
70,Maicon,D/WB (R),Unknown,Hong Kong Premier League,Hong Kong (China PR) (Premier League),BRA,186,82,37,Right,...,4,8,3,7,7,6,14,8,5,12
75,Ravil Tagir,D (C),AltÄ±nordu,Turkish 1. League,Turkey (1. League),TUR,182,70,16,Left,...,15,2,1,10,11,11,13,10,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45047,Zoran Josipovic,ST (C),Beroe,Bulgarian First League,Bulgaria (First League),SUI,191,82,23,Right,...,12,2,2,10,13,11,8,9,3,9
45048,Zoran Ratkovic,"AM (RC), ST (C)",Belisce,Croatian Third League East,Croatia (Third League East),CRO,175,70,40,Right,...,10,2,1,9,14,11,10,9,2,7
45049,Zsolt GÃ©vay,D (LC),Paks,Hungarian Division I,Hungary (Division I),HUN,187,80,31,Left,...,12,3,2,13,11,12,9,13,3,10
45050,Zurab Tsiskaridze,"D (LC), DM",Al-Jabalain,Saudi First Division,Saudi Arabia (First Division),GEO,185,85,32,Left Only,...,9,4,3,17,11,12,7,16,3,11
